In [8]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 3.5MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 3.5MB/s 


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

In [18]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car"


/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_two/dw_matrix_car


In [19]:

!ls


cars_visualisation.ipynb  data	day3_simple_model.ipynb  LICENSE  README.md


## Reading data

In [20]:
df = pd.read_hdf('data/car.h5')
df.shape

(106494, 155)

In [23]:
df.columns.values

array(['breadcrumb', 'created_at', 'price_currency', 'price_details',
       'price_value', 'seller_address', 'seller_name', 'seller_type',
       'feature_czujniki-parkowania-przednie',
       'feature_poduszka-powietrzna-chroniąca-kolana',
       'feature_kurtyny-powietrzne', 'feature_klimatyzacja-dwustrefowa',
       'feature_światła-led', 'feature_czujnik-zmierzchu',
       'feature_elektrycznie-ustawiane-lusterka',
       'feature_asr-(kontrola-trakcji)',
       'feature_poduszka-powietrzna-kierowcy', 'feature_cd',
       'feature_elektryczne-szyby-przednie',
       'feature_poduszka-powietrzna-pasażera',
       'feature_system-start-stop', 'feature_światła-do-jazdy-dziennej',
       'feature_komputer-pokładowy', 'feature_elektryczne-szyby-tylne',
       'feature_klimatyzacja-manualna', 'feature_tapicerka-welurowa',
       'feature_czujnik-deszczu', 'feature_światła-przeciwmgielne',
       'feature_ogrzewanie-postojowe', 'feature_radio-niefabryczne',
       'feature_regulowane-zaw

## Dummy Model

In [24]:
df. select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [31]:
  feats = ['car_id']
  X = df[ feats ].values
  y = df['price_value'].values

  model = DummyRegressor()
  model.fit(X, y)
  y_pred = model.predict(X)

  mae(y, y_pred)

39465.934630440985

In [32]:
# Remove prices in currencies different than PLN
df = df[ df['price_currency'] == 'PLN' ]
df.shape


(106290, 155)

In [0]:
SUFIX_CAT = '_cat'
  
for feat in df.columns:

  if isinstance(df[feat][0], list): continue

  factorized_values = df[ feat ].factorize()[0]

  if SUFIX_CAT in feat: 
    df[feat] = factorized_value
  else:  
    df[feat + SUFIX_CAT] = factorized_values

In [45]:
cat_feats = [x for x in df.columns if SUFIX_CAT in x]
cat_feats = [x for x in cat_feats if 'price' not in x]
len(cat_feats)


151

In [47]:

X = df[ cat_feats ].values
y = df['price_value'].values

model = DecisionTreeRegressor(max_depth=5)
scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')

np.mean(scores)

#model.fit(X, y)
#y_pred = model.predict(X)
#mae(y, y_pred)

-19566.588937368328

In [49]:

m = DecisionTreeRegressor(max_depth=5)
m.fit(X, y)

imp = PermutationImportance(m, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd_cat
0.2008 ± 0.0047,param_faktura-vat_cat
0.1943 ± 0.0088,param_stan_cat
0.1423 ± 0.0086,param_rok-produkcji_cat
0.0629 ± 0.0047,param_moc_cat
0.0424 ± 0.0014,feature_kamera-cofania_cat
0.0412 ± 0.0008,param_skrzynia-biegów_cat
0.0286 ± 0.0037,param_marka-pojazdu_cat
0.0191 ± 0.0022,param_pojemność-skokowa_cat
0.0163 ± 0.0005,feature_bluetooth_cat
